### Нормализация данных

Загрузим датасет с образцами здоровой и раковой ткани. Датасет состоит из 569 примеров, где каждой строчке из 30 признаков, соответствует класс `1` злокачественной (*malignant*) или `0` доброкачественной (*benign*) ткани. Задача состоит в том, чтобы по 30 признакам обучить модель определять тип ткани (злокачественная или доброкачественная).

Можно иметь сколь угодно хороший алгоритм для классификации - но до тех пор, пока данные на входе - мусор, на выходе из нашего чудесного классификатора мы тоже будем получать мусор (*garbage in - garbage out*). Давайте разберемся, что конкретно надо сделать, чтобы kNN реально заработал.


In [ ]:
import sklearn.datasets

cancer = sklearn.datasets.load_breast_cancer() # load data
X = cancer.data # features
Y = cancer.target # labels(classes)
print(f'X shape: {X.shape}, Y shape: {Y.shape}') 
print(f'X[0]: \n {X[0]}') 
print(f'Y[0]: \n {Y[0]}') 

Посмотрим сколько данных в классе `0` и сколько данных в классе `1`

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5)) # set fig size 
plt.bar(1,Y[Y==1].shape, label=cancer.target_names[0]) # 1 label 
plt.bar(0,Y[Y==0].shape, label=cancer.target_names[1]) # 0 label
plt.title('Class balance') 
plt.ylabel('Num examples') 
plt.xticks(ticks=[1,0], labels=['1','0']) 
plt.legend(loc='upper left') 
plt.show() 

Теперь давайте посмотрим на сами данные. У нас есть 569 строк в каждой из которой по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд)

In [ ]:
import pandas as pd
pd.DataFrame(X).describe()

То же самое, но в виде графика. Видно, что у фич совершенно разные значения.

In [ ]:
import seaborn as sns

ax = sns.boxenplot(data=pd.DataFrame(X), orient="h", palette="Set2")
ax.set(xscale='log', xlim=(1e-4, 1e4), xlabel='Values', ylabel='Features')
plt.show()

Чтобы адекватно сравнить данные между собой нам следует использовать нормализацию.

 **Нормализация, выбор Scaler**

Нормализация — это преобразование данных к неким безразмерным единицам.
Ключевая цель нормализации — приведение различных данных в самых разных единицах измерения и диапазонах значений к единому виду, который позволит сравнивать их между собой.

Главное условие правильной нормализации — все признаки должны быть равны в возможностях своего влияния.

Например, у нас есть данные по группе людей: *возраст* (в годах) и *размер дохода* (в рублях). Возраст может измениться в диапазоне от 18 до 70 ( интервал 70-18 = 52). А доход от 30 000 р до 500 000 р (интервал 500 000 - 30 000 = 470 000). В таком варианте разница в возрасте имеет меньшее влияние, чем разница в доходе. Получается, что доход становится более важным признаком, изменения в котором влияют больше при сравнении схожести двух людей.

Должно быть так, чтобы максимальные изменения любого признака в «основной массе объектов» были одинаковы. Тогда потенциально все признаки будут равноценны.

Осталось определиться с выбором инструмента, часто используют следующие варианты: `MinMaxScaler`, `StandardScaler`, `RobustScaler`

Сравним `MinMaxScaler`, `StandardScaler`, `RobustScaler` для признака `data[:,0]`. **Обратите внимание на ось X**

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np

np.random.seed(42)  # setting the initialization parameter for random values

# generate random values from 1 to 255, shape (30,1)
test = X[:,0].reshape(-1,1)

plt.figure(1, figsize=(30, 5))  
plt.subplot(141)  # set location
plt.scatter(test, range(len(test)), c=Y)  
plt.ylabel("Num examples", fontsize=15)  
plt.xticks(fontsize=15)  
plt.yticks(fontsize=15)  
plt.title("Non scaled data", fontsize=18)  

# scale data with MinMaxScaler
test_scaled = MinMaxScaler().fit_transform(test)  
plt.subplot(142)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("MinMaxScaler", fontsize=18)

# scale data  with StandardScaler
test_scaled = StandardScaler().fit_transform(test)  
plt.subplot(143)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("StandardScaler", fontsize=18)

# scale data  with RobustScaler
test_scaled = RobustScaler().fit_transform(test)  
plt.subplot(144)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("RobustScaler", fontsize=18)
plt.show()

Идея **`MinMaxScaler`** заключается в том, что он преобразует данные в диапазоне от 0 до 1. Может быть полезно, если нужно выполнить преобразование, в котором отрицательные значения не допускаются (e.g., масштабирование RGB пикселей)


$$z=\frac{X_i-X_{min}}{X_{max}-X_{min}}$$

$X_{min}$ и $X_{max}$ задаются как минимальное и максимальное допустимое значение, по умолчанию:  $X_{min}=0$  и $X_{max}=1$

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение 0 и стандартное отклонение 1. Большинство значений будет в  диапозоне от -1 до 1. Это стандартная трансформация, и она применима во многих ситуациях.

$$z=\frac{X-u}{s}$$

$u$ — среднее значение (или 0 при `with_mean=False`) и $s$ — стандартное отклонение (или 0 при `with_std=False`)

И StandardScaler и MinMaxScaler очень чувствительны к наличию выбросов. **`RobustScaler`** использует медиану и основан на *процентилях*. Процентиль — мера, в которой процентное значение общих значений равно этой мере или меньше ее. Например, 90 % значений данных находятся ниже 90-го процентиля, а 10 % значений данных находятся ниже 10-го процентиля. Соответственно, RobustScaler не зависит от небольшого числа очень больших предельных выбросов (outliers). Следовательно, результирующий диапазон преобразованных значений признаков больше, чем для предыдущих скэйлеров и, что более важно, примерно одинаков.

$$z=\frac{X-X_{median}}{IQR}$$

$X_{median}$ — значение медианы, $IQR$ — межквартильный диапазон равный разнице между 75-ым и 25-ым процентилями

Для нашей задачи по определению раковых опухолей обработаем наши 30 признаков с помощью StandardScaler.

In [ ]:
X_norm = StandardScaler().fit_transform(X)  # scaled data

Видим что они стали намного более сравнимы между собой.

In [ ]:
pd.DataFrame(X_norm).describe()

In [ ]:
ax = sns.boxenplot(data=pd.DataFrame(X_norm), 
                   orient="h", 
                   palette="Set2")
ax.set(xlabel='Values', ylabel='Features')
plt.show()

### Переобучение

Теперь обучим kNN для общей выборки данных, при разном значении количества соседей.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

n_nei_rng = np.arange(1, 31)  # array of the number of neighbors

quality = np.zeros(
    n_nei_rng.shape[0]
)  

for ind in range(n_nei_rng.shape[0]):  # for all elements
    # create knn for all num neighbors 
    knn = KNeighborsClassifier(
        n_neighbors=n_nei_rng[ind]
    )  
    knn.fit(X_norm, Y)  
    q = accuracy_score(y_pred=knn.predict(X_norm), y_true=Y)  # accuracy
    quality[ind] = q  # fill quality

plt.figure(figsize=(8, 5))  
plt.title("KNN on train", size=20)  
plt.xlabel("Neighbors", size=15)  
plt.ylabel("Accuracy", size=15)  
plt.plot(n_nei_rng, quality)  
plt.xticks(n_nei_rng) 
plt.show()  

Видим, что качество на 1 соседе - самое лучшее. Но это и понятно - ближайшим соседом элемента из обучающей выборки будет сам объект. Мы просто **запомнили** все объекты.

Если теперь мы попробуем взять какой-то новый образец опухоли и классифицировать его - у нас скорее всего ничего не получится. В таких случаях мы говорим, что наша модель не умеет обобщать (*generalization*).

Для того, чтобы знать заранее обобщает ли наша модель или нет, мы можем разбить все имеющиеся у нас данныe на 2 части. Но одной части мы будем обучать классификатор (*train set*), а на другой тестировать насколько хорошо он работает (*test set*).

In [ ]:
from sklearn.model_selection import train_test_split

# split data to train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train_norm = scaler.transform(X_train)  # scaling data
X_test_norm = scaler.transform(X_test)  # scaling data

n_nei_rng = np.arange(1, 31)  
train_quality = np.zeros(n_nei_rng.shape[0])  # quality on train data
test_quality = np.zeros(n_nei_rng.shape[0])  # quality on test data

for ind in range(n_nei_rng.shape[0]):  
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])  
    knn.fit(X_train_norm, Y_train)  
    
    # accuracy on train data
    trq = accuracy_score(y_pred=knn.predict(X_train_norm), y_true=Y_train)  
    train_quality[ind] = trq  

    # accuracy on test data
    teq = accuracy_score(y_pred=knn.predict(X_test_norm), y_true=Y_test)  
    test_quality[ind] = teq  

# accuracy plot  on train and test data
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Вот, теперь мы видим, что 1 сосед был "ложной тревогой". Такие случаи мы называем *переобучением*. Чтобы действительно предсказывать что-то полезное нам надо выбирать число соседей начиная минимум с 3

## Support Vector Machine (метод опорных векторов)

[Отличное видео про SVM от Stat Quest которое все объясняет](https://www.youtube.com/watch?v=efR1C6CvhmE&ab_channel=StatQuestwithJoshStarmer)

### 1D классификация

Рассмотрим одномерный пример. У нас есть данные по массе мышей. Часть из них определена как нормальные, а часть как мыши с ожирением. Чтобы их отделить друг от друга, нам достаточно одного критерия. Мы можем посмотреть на график, и визуально определить предельную массу, после которой мышки будут жирненькими

In [ ]:
import seaborn as sns

def generate_data(total_len=40):
    X = np.hstack([np.random.uniform(14,21, total_len//2), 
                   np.random.uniform(24,33, total_len//2)])
    Y = np.hstack([np.zeros(total_len//2), 
                   np.ones(total_len//2)])
    return X,Y

def plot_data(X, Y, total_len=40, s=50, threshold=21.5):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=Y, s=s)
    ax.axvline(threshold, color='red', ls='dashed')

    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g');
    return ax

total_len = 40
X, Y = generate_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)

Теперь пользуясь нашим простым критерием, попробуем классифицировать каких-то новых мышей

In [ ]:
X_test = np.random.uniform(14,30, 5)

def classify(X, threshold=21.5):
    y = np.zeros_like(X)
    y[X > threshold] = 1
    return y

total_len = 40
X, Y = generate_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)
ax = plot_data(X_test, classify(X_test), total_len=total_len, s=300)

Но что если наши мыши находятся тут?


In [ ]:
X_test = np.array([21.45, 22.5])

total_len = 40
X, Y = generate_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)
ax = plot_data(X_test, classify(X_test), total_len=total_len, s=300)

С точки зрения нашего классификатора - все четко. Больше порогового значения - значит перевес, меньше - значит нормальные. Но с точки зрения здравого смысла, логичнее было бы классифицировать обоих мышей как нормальных, так как они значительно ближе к нормальным, чем к ожиревшим.

Вооружившись этим новым знанием, попробуем классифицировать наших отъевшихся мышек по-умному. Возьмем крайние точки в каждом класстере. И в качестве порогового значения будем использовать среднее между ними

In [ ]:
X, Y = generate_data(total_len=total_len)
normal_limit = X[Y==0].max() # extreme point for 'normal'
obese_limit = X[Y==1].min() # extreme point for 'obese'

threshold = np.mean([normal_limit, obese_limit]) # separated with mean value 

X_test = np.array([21.5, 23])
ax = plot_data(X, Y, total_len=total_len, threshold=threshold)
ax = plot_data(X_test, classify(X_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold)

Мы можем посчитать, насколько наша мышь близка к тому, чтобы оказаться в другом классе. Такое расстояние называется **margin**. И оно считается как $\mathrm{margin} = |\mathrm{threshold} - \mathrm{observation}|$

In [ ]:
margins = np.abs(X_test - threshold)
print(margins)

Соответственно, если мы посчитаем margins для наших крайних точек `normal_limit` и `obese_limit`, мы найдем самое большое возможное значение margin для нашего классификатора

In [ ]:
margin_0 = np.abs(normal_limit - threshold)
margin_1 = np.abs(obese_limit - threshold)
print(margin_0, margin_1)

Такой классификатор, мы называем **Maximum Margin Classifier**. Он хорошо работает в случае, когда все данные размечены аккуратно. Теперь рассмотрим более реалистичный пример, где что-то пошло не так

In [ ]:
def generate_realistic_data(total_len=40):
    X = np.hstack([np.random.uniform(14,21, total_len//2), np.random.uniform(24,33, total_len//2)])
    Y = np.hstack([np.zeros(total_len//2), np.ones(total_len//2)])
    indx = np.where(X == X[Y==1].min())[0]
    Y[indx] = 0
    s = np.ones_like(X)*50
    s[indx] = 300
    return X,Y,s

total_len = 40
X,Y,s = generate_realistic_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len, s=s)

В таком случае, наш **Maximum Margin Classifier** работать не будет. Исходя из этого, мы можем прийти к выводу, что наш классификатор очень чувствителен к выбросам. Давайте подумаем можно ли это как-то исправить?

Например, мы можем разрешить нашему классификатору ошибаться. Если мы будем использовать в качестве порогового значения не самое крайнее, а следующее за ним - мы промажем в классификации конкретно этой странной точки. Но в целом будем правы. Margin определенный таким образом, называется **Soft margin**.

In [ ]:
total_len = 40
X, Y, s = generate_realistic_data(total_len=total_len)

normal_limit = np.sort(X[Y==0])[-2]
obese_limit = np.sort(X[Y==1])[1]

threshold = np.mean([normal_limit, obese_limit])

X_test = np.array([20.5, 25])
ax = plot_data(X, Y, total_len=total_len, threshold=threshold)
ax = plot_data(X_test, classify(X_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold)

Но почему мы решили взять именно следующее значение? Почему не через 2? Откуда мы знаем что это лучший из возможных вариантов? А ни откуда не знаем. Чтобы узнать какой из margins лучше, нам стоит численно это проверить и посчитать, сколько раз мы ошибемся, если возьмем в качестве порогового значения между каждой парой точек. Для этого мы вновь воспользуемся *кросс-валидацией*.

In [ ]:
total_len = 40
X, Y, s = generate_realistic_data(total_len=total_len)

indxs = []
accuracies = []
thresholds = []

X_test, Y_test = generate_data(total_len=total_len)

for i in range(total_len//2):
    for j in range(total_len//2-1):
        normal_limit = np.sort(X[Y==0])[-i]
        obese_limit = np.sort(X[Y==1])[j]

        threshold = np.mean([normal_limit, obese_limit])
        
        Y_pred = classify(X_test, threshold=threshold)
        accuracy = np.mean(Y_test == Y_pred)
        indxs.append((-i,j))
        accuracies.append(accuracy)
        thresholds.append(threshold)

print(f'Accuracy = {np.max(accuracies)*100}%')
print(f'Best indexes', indxs[np.argmax(accuracies)])

best_treshold = thresholds[np.argmax(accuracies)]
print('Best treshhold value %.2f'% best_treshold)

ax = plot_data(X_test, classify(X_test, threshold=best_treshold), 
               total_len=total_len, 
               s=200, 
               threshold=best_treshold)

Когда для классификатора используется **Soft Margin** - такой классификатор называют **Soft Margin Classifier** или по другому - **Support Vector Classifier**. По сути это уже SVM. 

### 2D класификация

Теперь рассмотрим пример, где мы измерили не только вес мышей, но и их длину от хвоста до носа. Мы можем вновь применить наш метод Support Vector Classifier, и теперь классы разделяет не одно пороговое значение (по сути, точка), а линия.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn import svm

def generate_2d_data(total_len=40):
    X, Y = make_blobs(n_samples=total_len, centers=2, random_state=42)
    X[:,0] += 10 
    X[:,1] += 20 
    return X, Y

def plot_data(X, Y, total_len=40, s=50, threshold=21.5):
    ax = sns.scatterplot(x=X[:,0], y=X[:,1], hue=Y, s=s)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g', ylabel='Length, cm');
    return ax

total_len = 40
X, Y = generate_2d_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)

# Code for illustration, later we will understand how it works
# fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel='linear', C=1000)
clf.fit(X, Y)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.show()

### 3D классификация

Если мы добавим еще одно измерение - возраст, мы обнаружим, что наши данные стали трехмерными, а разделяет их теперь не линия, а плоскость

In [ ]:
def generate_3d_data(total_len=40):
    X, Y = make_blobs(n_samples=total_len, centers=2, random_state=42, n_features=3)
    X[:,0] += 10 
    X[:,1] += 20 
    X[:,2] += 10
    return X, Y

def plot_data(X, Y, total_len=40, s=50, threshold=21.5):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs=X[:,0], ys=X[:,1], zs=X[:,2], c=Y, s=s, cmap='Set1')
    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    ax.plot_surface(XX, YY, XX*YY*0.2, alpha=0.2)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g', ylabel='Length, cm', zlabel='Age, days');
    return ax

total_len = 40
X, Y = generate_3d_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)


Соответственно, если бы у нас было 4 измерения и больше (например: вес, длинна, возраст, кровяное давление), то многомерная плоскость которая бы разделяла наши классы - называлась бы **гиперплоскость** (рисовать мы ее, конечно же, не будем). Чисто технически, и точка, и линия - тоже гиперплоскости. Но все же гиперплоскостью принято называть то, что нельзя нарисовать на бумаге.

### SVM

Данные не всегда разделяются так хорошо как в случае нашего мышиного датасета. Например, рассмотрим следующее: у нас есть данные по дозировке лекарства и 2 класса - пациенты, которые поправились, и те, которым лучше не стало

In [ ]:
def generate_patients_data(total_len=40):
    X = np.random.uniform(0,50, total_len)
    Y = np.zeros_like(X)
    Y[(X > 15) & (X < 35)] = 1
    return X, Y

def plot_data(X, Y, total_len=40, s=50):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=Y, s=s)

    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Recover', 'Sick'])
    ax.set(xlabel='dose, mg');
    return ax

total_len = 40
X, Y = generate_patients_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)

Соответственно мы не можем найти такое пороговое значение, которое будет разделять наши классы на больных и здоровых, а следовательно, и Support Vector Classifier работать тоже не будет.  Для начала, давайте преобразуем наши данные таким образом, что бы они стали 2-х мерными. В качестве значений по оси Y будем использовать дозу возведенную в квадрат (**доза**$^2$).

In [ ]:
def plot_data(X, Y, total_len=40, s=50):
    ax = sns.scatterplot(x=X[0,:], y=X[1,:], hue=Y, s=s)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Recover', 'Sick'])
    ax.set(xlabel='Dose, mg');
    ax.set(ylabel='Dose$^2$');
    return ax

total_len = 40
X_1, Y = generate_patients_data(total_len=total_len)
X_2 = X_1**2
X = np.vstack([X_1, X_2])

plot_data(X, Y, total_len=40, s=50)
plt.show()

Теперь мы можем вновь использовать Support Vector Classifier для классификации

In [ ]:
plot_data(X, Y, total_len=40, s=50)

x = np.linspace(0,50,50)
xs = [X[0,:][Y==1].min(), X[0,:][Y==1].max()]
ys = [X[1,:][Y==1].min(), X[1,:][Y==1].max()]

# Calculate the coefficients.
coefficients = np.polyfit(xs, ys, 1)

# Let's compute the values of the line...
polynomial = np.poly1d(coefficients)
y_axis = polynomial(x)

# ...and plot the points and the line
plt.plot(x, y_axis, 'r--')
plt.show()

Но тут возникает резонный вопрос - почему мы решили возвести в квадрат? Почему не в куб? Или наоборот не извлечь корень? Как нам решить какое преобразование использовать?

И у нас есть **вторая проблема** - а если перейти надо в пространство очень большой размерности? В этом случае наши данные очень сильно увеличатся в размере.

Комбинация двух проблем дает нам **много радости** - надо перебирать большое число возможных пространств большей размерности



Однако основная фишка Support Vector Machine состоит в том, что внутри он работает на скалярных произведениях. И можно эти скалярные произведения считать, **не переходя в пространство большей размерности**


Для этого SVM использует **Kernel Function**. 

Kernel Function может, например, быть полиномом (**Polynomial Kernel Function**), который имеет параметр $d$ - сколько размерностей выбрать. 

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L02/svm_kernel_function.png" width="500">

Примеры ядер :

* $k(x_i, x_j) = (<x_i, x_j> + c)^d, с, d \in \mathbb{R}$ - полиномиальное ядро, считает расстояние между объектами в пространстве размерности d

* $k(x_i, x_j) = \frac{1}{z} e^{-\frac{h(x_i, x_j)^2}{h}}$ - радиальная базисная функция RBF


Таким образом, в случае SVM можно легко перебрать много таких пространств на кроссвалидации и выбрать более удобное. 

Более того, SVM может проверять пространства признаков бесконечного размера. Если для такого пространства существует kernel function. Иногда такие пространства оказываются очень удобными для решения задач. Часто используют тот же RBF-пространство, приведенное выше. А оно как раз бесконечно-мерное.

##### $\color{blue}{\text{*Не обязательное задание:}}$

Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количество эпох до 20).

In [ ]:
import os
from IPython.display import clear_output

file_exists = os.path.exists("/content/cifar-10-batches-py")
if file_exists == False:
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    !tar -xzf cifar-10-python.tar.gz   
clear_output()

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


X_train = np.zeros((0, 3072))
Y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    X_train = np.append(X_train, np.array(raw[b"data"]), axis=0)
    Y_train = np.append(Y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
X_test = np.array(test[b"data"])
Y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train shape: {X_test.shape}, Y_train shape: {Y_test.shape}")

In [ ]:
import random

class LinearClassifier():
    def __init__(self, labels, batch_size, random_state=42):
        self.labels = labels  # classes names
        self.classes_num = len(labels)  # num of classes

        np.random.seed(
            random_state
        )  
        self.W = (
            np.random.randn(3073, self.classes_num) * 0.0001
        )  # generate random weights, reshape to add bias 
        self.batch_size = batch_size  # batch_size

    def fit(self, X_train, Y_train, learning_rate=1e-8):
        loss = 0.0  # обнуляем loss
        train_len = X_train.shape[0]  # num of examples
        indexes = list(range(train_len))  # indexes train_len
        random.shuffle(indexes)  

        for i in range(
            0, train_len, self.batch_size
        ):  
            idx = indexes[
                i : i + self.batch_size
            ]  # 
            X_batch = X_train[idx]  
            Y_batch = Y_train[idx]  

            X_batch = np.hstack(
                [X_batch, np.ones((X_batch.shape[0], 1))]
            )  # add bias

            loss_val, grad = self.loss(X_batch, Y_batch)  # loss and gradient
            self.W -= learning_rate * grad  # update weigths

            loss += loss_val  # loss sum
        return loss / (train_len)  # mean loss

    def loss(self, X, Y):
        current_batch_size = X.shape[0]  # batch_size
        loss = 0.0  
        dW = np.zeros(self.W.shape) 
        for i in range(current_batch_size):  
            scores = X[i].dot(
                self.W
            )  # vector of shape 10
            correct_class_score = scores[
                int(Y[i])
            ]  
            above_zero_loss_count = 0  
            for j in range(self.classes_num): 
                if j == Y[i]:  # predict class
                    continue
                margin = scores[j] - correct_class_score + 1  # loss
                if margin > 0:  
                    above_zero_loss_count += (
                        1  
                    )
                    loss += margin  # 
                    dW[:, j] += X[i]  # 
            dW[:, int(Y[i])] -= above_zero_loss_count * X[i]  
        loss /= current_batch_size  
        dW /= current_batch_size  
        return loss, dW

    def forward(self, X):
        X = np.append(X, 1)  # add 1 (bias)
        scores = X.dot(self.W)  
        return np.argmax(scores)  

In [ ]:
import time

def validate(model, X_test, Y_test, noprint=False):
    correct = 0  
    for i, img in enumerate(X_test):  
        index = model.forward(img)  
        correct += (
            1 if index == Y_test[i] else 0
        )  
        if noprint is False:  
            if i > 0 and i % 1000 == 0:  
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  
    return correct / len(Y_test)  

In [ ]:
print("How learning quality depends of speed:")

for lr in [1e-2, 1e-8]:
    for bs in [256, 2048]:

        print("-" * 50, "\n", "learning_rate =", lr, "\tbatch_size =", bs)
        print()
        lc_model = LinearClassifier(labels_eng, batch_size=bs)

        best_accuracy = 0
        for epoch in range(10):
            loss = lc_model.fit(X_train, Y_train, learning_rate=lr)
            accuracy = validate(lc_model, X_test, Y_test, noprint=True)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
            print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

        print()
        print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

###  Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$

### MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$




### Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 







In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

X = [i for i in range(-5, 6)]
Y = [abs(i) for i in range(-5, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, label="y = |x|")
plt.title("y = |x|", size=20)
plt.legend()
plt.show()

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1.

Если больше +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>


In [ ]:
X = [i for i in range(-5, 1, 1)]
Y = [i * 0 - 1 for i in range(6)]
X_1 = [i for i in range(0, 6)]
Y_1 = [i * 0 + 1 for i in range(0, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, "b")
plt.plot(X_1, Y_1, "b")
plt.plot(0, 0, "ro")
plt.plot(0, 1, "bo")
plt.plot(0, -1, "bo")
plt.show()

### Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если $x > y$, то он оказал влияние на $b$. Иначе, его вклада в $b$ НЕ БЫЛО - градиент равен 0

 


Из: 

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Получаем:

$ \nabla_W	L(W) = {1 \over N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)$



__Сложность модели__ (*model complexity*) &mdash; важный гиперпараметр. В частности, для линейной модели, сложность может быть представлена количеством параметров, для полиномиальных моделей &mdash; степенью полинома, для деревьев решений &mdash; глубиной дерева и т.д.

Сложность модели тесно связана с __ошибкой обобщения__ (_generalization error_). Ошибка обобщения отличается от ошибки обучения, измеряемой на тренировочных данных, тем, что позволяет оценить обобщающую способность модели, приобретенную в процессе обучения, давать точные ответы на неизвестных ей объектах. Cлишком простой модели не будет хватать мощности для обобщения сложной закономерности в данных, что приводит к большой ошибке обобщения, с другой стороны слишком сложная модель также приводит к большой ошибке обобщения за счет того, что в силу своей сложности модель начинает пытаться искать закономерности в шуме, добиваясь большей точности на тренировочных данных, теряя при этом часть обобщающей способности.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L02/model_complexity.png" width="500"> 

Проиллюстрируем описанное явление на примере полиномиальной модели.

In [ ]:
x = np.linspace(0, 2*np.pi, 10)
y = np.sin(x) + np.random.normal(scale=0.25, size=len(x))
plt.scatter(x, y, s=50, facecolors='none', edgecolors='b', label='noisy data')

x_true = np.linspace(0, 2*np.pi, 200)
y_true = np.sin(x_true)
plt.plot(x_true, y_true, c='lime', label='ground truth')
plt.legend()
plt.show()

Попробуем аппроксимировать имеющуюся зависимость с помощью полиномиальной модели, используя шумные данные в качестве тренировочных данных:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

x_train = x.reshape(-1,1)

fig = plt.figure(figsize=(12,6))

for i, degree in enumerate([0,1,3,9]):

    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

    model.fit(x_train, y)
    y_plot = model.predict(x_true.reshape(-1,1))

    fig.add_subplot(2,2,i+1)
    plt.plot(x_true, y_plot, c='red', label=f'M={degree}')
    plt.scatter(x, y, s=50, facecolors='none', edgecolors='b')
    plt.plot(x_true, y_true, c='lime')
    plt.legend()
plt.show()

Видно, что модель может переобучаться, подстраиваясь под тренировочную выборку. В полиноме степень, и как следствие количество весов &mdash; это гиперпараметр, который можно подбирать на кросс-валидации, однако, когда мы таким образом подбираем сложность модели мы налагаем довольно грубое ограничение на обобщающую способность модели в целом. Вместо этого более разумным было бы оставить модель сложной, но использвать некий ограничитель (__регуляризатор__), который будет заставлять модель отдавать предпочтение выбору более простого обобщения. 

In [ ]:
from sklearn.linear_model import Ridge

model = make_pipeline(PolynomialFeatures(9), LinearRegression())
model_ridge = make_pipeline(PolynomialFeatures(9), Ridge(alpha=0.1))

model.fit(x_train, y)
y_plot = model.predict(x_true.reshape(-1,1))

model_ridge.fit(x_train, y)
y_plot_ridge = model_ridge.predict(x_true.reshape(-1,1))

plt.plot(x_true, y_plot, c='red', label=f'M={degree}')
plt.plot(x_true, y_plot_ridge, c='black', label=f'M={degree}, alpha=0.1')
plt.scatter(x, y, s=50, facecolors='none', edgecolors='b')
plt.plot(x_true, y_true, c='lime', label='ground truth')
plt.legend()
plt.show()

poly_coef = model[1].coef_

eq = f'y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x'
for i in range(2, 10):
    eq += f'+{round(poly_coef[i], 2)}*x^{i}'
    
print('Without regularization: ', eq)

poly_coef = model_ridge[1].coef_

eq = f'y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x'
for i in range(2, 10):
    eq += f'+{round(poly_coef[i], 2)}*x^{i}'

print('With regularization: ', eq)

Видно, что одним из "симптомов" переобучения являются аномально большие веса. Модель Ridge Regression, показанная в примере выше, использует L2 регуляризуцию для борьбы с этим явлением.




<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/l2_regularization.jpg" width="700"> 


L2 Regularization = weights decay

Идея состоит в том, что мы можем наложить некоторое требование на сами веса. Дело в том, что можно получить один и тот же выход модели при разных весах (выход модели соответствует умножению весов на $x$), при разных $w$ выход может быть идентичен.

Эти параметры задают некоторую аппроксимацию нашей целевой функции, аппроксимировать функцию можно двумя способами:
1. Использовать все имеющиеся данные и провести ее строго через все точки, которые нам известны; 
2. Использовать более простую функцию: (в данном случае линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым общим закономерностям, которые у них есть. 

Характерной чертой переобучения является второй сценарий, и сопровождается он, как правило, большими весами. Введение L2-регуляризации приводит к тому, что большие веса штрафуются и предпочтение отдается решениям, использующим малые значения весов. 

Модель может попробовать схитрить и по-другому - использовать все веса, все признаки, даже незначимые, но с маленькими коэффициентами. С этим L2-loss поможет хуже, так как он не сильнее штрафует мелкие веса. Результатов его применения - малые значения весов, которые использует модель

В этом случае на помощь приходит L1-loss, который штрафует вес за сам факт отличия его от нуля. Но и штрафует он все веса одинаково. Результат его применения - малое число весов, которые использует модель в принципе. 



<img src="https://edunet.kea.su/repo/EduNet-content/L02/img_license/l1_and_l2_regularization.gif" alt="alttext" style="width: 450px;"/>


Это лоссы можно комбинировать - получится Elastic Net




$\lambda=$ regularization strength (hyperparameter)

$L(W)=\underbrace{\frac1N\sum_{i=1}^NL_i(f(x_i,W),y_i)}_{\textbf{Data loss} }+\underbrace {\lambda R(W)}_{\textbf{Regularization}}$

Берем сумму всех весов по всей матрице $w$, и добавляем ее к loss. Соответственно, чем больше будет эта сумма, тем больше будет суммарный loss. 


В дальнейшем проблема с переобучением будет вставать довольно часто. Методов регуляризации модели существует достаточно много. Этот — один из базовых, который будет использоваться практически во всех оптимизаторах, с которыми познакомимся позже.

Посмотрим на графиках. Возьмем массив случайных логитов и применим к ним softmax

In [ ]:
rand_logits = np.linspace(-1,1,50)
fig,ax = plt.subplots(ncols=2)

ax[0].plot(np.arange(50), rand_logits)
ax[0].set_title('Logits')
ax[1].plot(np.arange(50), softmax(rand_logits))
ax[1].set_title('Softmax')
plt.show()



Начнем с простого примера, пусть у нас есть 10 точек со следующими значениями признака x: 

`x = [-2.2, -1.4, -0.8, 0.2, 0.4, 0.8, 1.2, 2.2, 2.9, 4.6]`

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L02/cross_entropy_ten_dots.png">

Пусть наши точки принадлежат двум классам: зеленый и красный:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L02/cross_entropy_two_classes.png">

Перед нами простая задача классификации: по признаку x предсказать класс наших точек. Мы можем переформулировать задачу как нахождение вероятности того, что точка зеленая или красная. В идеальной ситуации для зеленой точки вероятность того, что она зеленая равна 1, в то же время вероятность того, что красная точка &mdash; зеленая должна быть равна 0. 

Вероятности, которые выдает, обучаемая нами модель, зачастую далеки от идеальной ситуации. В нашем примере сравнить насколько сильно предсказанная вероятность класса отличается от вероятности, выдаваемой "идеальной моделью" можно за счет __бинарной кросс-энтропии__ частного случая кросс-энропии.

$$H_p(q)=-\frac{1}{N}\sum^N_{i=1}y_i\cdot log(p(y_i))+(1-y_i)\cdot log(1-p(y_i))$$

где $y$ &mdash; метка класса (1 для зеленого, 0 для красного), которую можно также интерпретировать как вероятность, предсказанную "идеальной моделью", $p(y)$ &mdash; вероятность того, что точка зеленая, предсказываемая оцениваемой моделью.

Какое же отношение энтропия имеет к этой формуле? Давайте углубимся в детали.

Поскольку $y$ представляет метку классов точек, то его распределение $q(y)$ выглядит следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L02/cross_entropy_distribution.png">

__Энтропия__ &mdash; мера неуверенности, связанная с распределением $q(y)$. Какова была бы мера неуверенности распределения $q(y)$ если бы все точки были зелеными? Так как у нас бы не было сомнений насчет цвета точки (он всегда зеленый), значение энтропии было бы 0. Теперь представим другую ситуацию, пусть у нас поровну точек зеленого и красного цвета. Для нас это наихудшая ситуация, поскольку попытка определить цвет точки по сути представляет случайное угадывание. В этом случае энтропия вычисляется по формуле Хартли:

$$H(q)=log(2)$$

Мы рассмотрели два крайних случая, но как быть с промежуточными ситуациями? Для этих случаев мы можем использовать формулу Шеннона:

$$H(q)=-\sum^C_{c=1}q(y_c)\cdot log(q(y_c))$$

где C &mdash; количество классов. Нетрудно заметить, что формула Хартли является частным случаем формулы Шеннона.

Таким образом, зная истинное распределение случайной величины, мы можем рассчитать его энтропию. А что будет если мы попытаемся аппроксимировать истинное распределение $q(y)$ некоторым другим распределением $p(y)$? Допустим, что наши цветные точки подчиняются этому распределению $p(y)$, также мы знаем, что исходят они из неизвестного нам истинного распределения $q(y)$, если мы посчитаем следующую энтропию, это и будет __кросс-энтропия__:

$$H_p(q)=-\sum^C_{c=1}q(y_c)\cdot log(p(y_c))$$

Если окажется, что распределения $p(y)$ и $q(y)$ совпадают, в этом случае энтропия $H(q)$ и кросс-энтропия $H_p(q)$ также будут совпадать. Однако в реальности такое случается редко и кросс-энтропия бывает больше энтропии истинного распределения

$$H_p(q)-H(q)\geq0$$

Разница между кросс-энтропией и энтропией называется __дивергенцией Кульбака-Лейблера__, которая является мерой различия между двумя распределениями:

$$D_{KL}(q||p)=H_p(q)-H(q)=\sum^C_{c=1}q(y_c)\cdot [log(q(y_c))-log(p(y_c))]$$

Это значит, что чем ближе $p(y)$ к $q(y)$, тем меньше будет значение дивергенции Кульбака-Лейблера и, следовательно, меньше значение кросс-энтропии.

Таким образом, мы хотим добиться, чтобы модель, которую мы оцениваем порождала $p(y)$ близкое к $q(y)$. Для этих целей мы стремимся __минимизировать кросс-энтропию__.

